In [6]:
from langdetect import detect
import pandas as pd
import re
from glob import glob
from tqdm.auto import tqdm

In [9]:


def detect_language(text):
    try:
        return detect(text)
    except:
        return None

def parse_text(text):
    pattern = r"<(.*?)>\n(.*?)(?=\n<|$)"

    # Find all matches
    matches = re.findall(pattern, text, re.DOTALL)

    # Process matches
    processed_matches = []
    for match in matches:
        character = match[0]
        lines = match[1].strip().split("\n")
        japanese_text = []
        english_text = []

        for line in lines:
            if detect_language(line) == 'ja':
                japanese_text.append(line)
            else:
                english_text.append(line)

        processed_matches.append((character, " ".join(japanese_text), " ".join(english_text)))

    # Convert to DataFrame
    df = pd.DataFrame(processed_matches, columns=['Character', 'Japanese', 'English'])
    return df



In [12]:
# load the game script file and convert them to dataframes
df_list = []
for fn in tqdm(glob('game_scripts/dgs1/*.txt')):
    with open(fn) as f:
        text = f.read()
        df_list.append(parse_text(text))

df_scripts = pd.concat(df_list)
        

  0%|          | 0/422 [00:00<?, ?it/s]

100%|██████████| 422/422 [03:03<00:00,  2.30it/s]


In [15]:
df_scripts = df_scripts.reset_index()
df_scripts.to_excel('game_scripts/dgs1.xlsx')